In [49]:
import sys, json, time, os
sys.path.append(sys.path.append(os.path.join(os.getcwd(), '..')))
from datamart.index_builder import IndexBuilder
from datamart.query_manager import QueryManager

Create es index

In [50]:
es_index = "datamart_tmp"

qm = QueryManager()

if qm.check_exists(index=es_index):
    qm.delete_index(index=[es_index])
qm.create_index(index=es_index)

In [55]:
tmp_description = "tmp/tmp.json"
tmp_out = "tmp/tmp_metadata.out"

A sample dastaset schema file for NOOA

In [56]:
print(json.dumps(json.load(open(tmp_description)), indent=2))

{
  "title": "TAVG",
  "description": "Average temperature (tenths of degrees C)[Note that TAVG from source 'S' corresponds to an average for the period ending at 2400 UTC rather than local midnight]",
  "url": "https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt",
  "keywords": [
    "Average",
    "cloudiness",
    "midnight",
    "to",
    "midnight",
    "from",
    "30-second",
    "ceilometer",
    "data"
  ],
  "provenance": "noaa.org",
  "materialization": {
    "python_path": "nooa_materializer",
    "arguments": {
      "type": "TAVG"
    }
  },
  "variables": [
    {
      "name": "date",
      "description": "the date of data",
      "semantic_type": [
        "https://metadata.datadrivendiscovery.org/types/Time"
      ],
      "temporal_coverage": {
        "start": "1994-03-19T00:00:00",
        "end": "1996-05-28T00:00:00"
      }
    },
    {
      "name": "stationId",
      "description": "the id of station which has this data",
      "semantic_type": [
        "

Create metadata

In [57]:
index_builder = IndexBuilder()
this_metadata = index_builder.indexing(description_path=tmp_description,
                                       data_path=None,
                                       query_data_for_indexing=True)

A sample metadata for NOOA

In [58]:
print(json.dumps(this_metadata, indent=2))

{
  "datamart_id": 0,
  "title": "TAVG",
  "description": "Average temperature (tenths of degrees C)[Note that TAVG from source 'S' corresponds to an average for the period ending at 2400 UTC rather than local midnight]",
  "url": "https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt",
  "keywords": [
    "Average",
    "cloudiness",
    "midnight",
    "to",
    "midnight",
    "from",
    "30-second",
    "ceilometer",
    "data"
  ],
  "date_published": null,
  "date_updated": null,
  "provenance": "noaa.org",
  "original_identifier": null,
  "materialization": {
    "python_path": "nooa_materializer",
    "arguments": {
      "type": "TAVG"
    }
  },
  "variables": [
    {
      "datamart_id": 1,
      "name": "date",
      "description": "the date of data",
      "semantic_type": [
        "https://metadata.datadrivendiscovery.org/types/Time"
      ],
      "temporal_coverage": {
        "start": "1994-03-19T00:00:00",
        "end": "1996-05-28T00:00:00"
      }
    },
    

In [59]:
qm.create_doc(index='datamart_tmp', doc_type='document', body=this_metadata, id=this_metadata['datamart_id'])
time.sleep(1)

In [60]:
query_1 = json.dumps({
    "query": {
        "bool": {
            "must": [
                {
                    "match": {
                        "variables.named_entity": "los angeles"
                    }
                }
            ]
        }
    }
})

In [61]:
hitted_metadatas = qm.search(index=es_index, body=query_1)

print(len(hitted_metadatas))

1


Materialize data

In [62]:
for metadata in hitted_metadatas:
    print("====== HIT a metadata ======")
    print(json.dumps(metadata, indent=2))
    print("\n\n")
    print("====== GET the dataset ======")
    df = qm.get_dataset(metadata=metadata)
    print(df.iloc[:10, :])
    print("\n\n")

====== HIT a metadata ======
{
  "datamart_id": 0,
  "title": "TAVG",
  "description": "Average temperature (tenths of degrees C)[Note that TAVG from source 'S' corresponds to an average for the period ending at 2400 UTC rather than local midnight]",
  "url": "https://www1.ncdc.noaa.gov/pub/data/ghcn/daily/readme.txt",
  "keywords": [
    "Average",
    "cloudiness",
    "midnight",
    "to",
    "midnight",
    "from",
    "30-second",
    "ceilometer",
    "data"
  ],
  "date_published": null,
  "date_updated": null,
  "provenance": "noaa.org",
  "original_identifier": null,
  "materialization": {
    "python_path": "nooa_materializer",
    "arguments": {
      "type": "TAVG"
    }
  },
  "variables": [
    {
      "datamart_id": 1,
      "name": "date",
      "description": "the date of data",
      "semantic_type": [
        "https://metadata.datadrivendiscovery.org/types/Time"
      ],
      "temporal_coverage": {
        "start": "1994-03-19T00:00:00",
        "end": "1996-05-28T

                  date          stationid         city TAVG
0  2018-09-23T00:00:00  GHCND:USR0000CACT  los angeles  233
1  2018-09-23T00:00:00  GHCND:USR0000CBEV  los angeles  206
2  2018-09-23T00:00:00  GHCND:USR0000CCHB  los angeles  228
3  2018-09-23T00:00:00  GHCND:USR0000CCHI  los angeles  218
4  2018-09-23T00:00:00  GHCND:USR0000CCLE  los angeles  237
5  2018-09-23T00:00:00  GHCND:USR0000CCP9  los angeles  224
6  2018-09-23T00:00:00  GHCND:USR0000CLTU  los angeles  215
7  2018-09-23T00:00:00  GHCND:USR0000CMAL  los angeles  197
8  2018-09-23T00:00:00  GHCND:USR0000CMIL  los angeles  212
9  2018-09-23T00:00:00  GHCND:USR0000CSFD  los angeles  222



